In [1]:
import numpy as np

In [16]:
GRAVITY = 9.81
MASS = 1.5
GRAVITY = 9.806
T_LOOKAHEAD = 0.8

In [26]:
A = np.array([
                [0, 0, 0,    1, 0, 0,     0,   0, 0],
                [0, 0, 0,    0, 1, 0,     0,   0, 0],
                [0, 0, 0,    0, 0, 1,     0,   0, 0],

                [0, 0, 0,    0, 0, 0,     0,-1*GRAVITY, 0],
                [0, 0, 0,    0, 0, 0,     GRAVITY,   0, 0],
                [0, 0, 0,    0, 0, 0,     0,   0, 0],

                [0, 0, 0,    0, 0, 0,     0,   0, 0],
                [0, 0, 0,    0, 0, 0,     0,   0, 0],
                [0, 0, 0,    0, 0, 0,     0,   0, 0],

            ])
A

array([[ 0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,
         0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,
         0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -9.806,
         0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  9.806,  0.   ,
         0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ]])

In [27]:
B = np.array([
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0],

                [-1/MASS, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1],
            ])
B

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [-0.66666667,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [28]:
C = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 1, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 1, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [29]:
x = np.array([[0., 0., -.5, 0., 0., 0., 0., 0., 0.]]).T
u = np.array([[16., 0., 0., 0.]]).T

In [32]:
eAT = np.exp2(A*T_LOOKAHEAD)

In [38]:
import sympy as smp
import math as m

import scipy.integrate as sp_int
import scipy.linalg as sp_linalg

In [79]:
def getyorai_gJac_linear_predict(int_eATB): #Calculates Jacobian of Linearized System Output Prediction for Newton-Raphson Input Calc : udot = a*inv(dg/du)(err)
    """ Calculates the Jacobian of the closed form linearized system output prediction wrt inputs at the operating point of hover above origin with euler angles 0. """
    Jac = np.concatenate((int_eATB[0:3, :], int_eATB[-1:, :]), axis=0) # dg/du = C * int_eATB
    print(f"{Jac =}")
    return Jac

def observer_matrix(): #Calculates Observer Matrix for Prediction of desired outputs from all 9 states
    """ Calculates the observer matrix for prediction of desired outputs from all 9 states. """
    C = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 1]])
    return C

def linearized_model(): #Calculates Linearized Model Matrices for Prediction (eAT, int_eATB, int_eAT, C)
    """ Calculates the linearized model matrices for prediction. """

    # x(t) = eAT*x(t0) + int_eATB*u(t) with u(t) = u over T = T_lookahead seconds
    # simplifies to x(t) = eAT*x(t) + int_eATB*(u - gravity) in our implementation as seen in getyorai_g_linear_predict
    # y(t) = C*x(t)

    A = smp.Matrix(
        [
            [0, 0, 0,    1, 0, 0,     0,   0, 0],
            [0, 0, 0,    0, 1, 0,     0,   0, 0],
            [0, 0, 0,    0, 0, 1,     0,   0, 0],

            [0, 0, 0,    0, 0, 0,     0,-1*GRAVITY, 0],
            [0, 0, 0,    0, 0, 0,     GRAVITY,   0, 0],
            [0, 0, 0,    0, 0, 0,     0,   0, 0],

            [0, 0, 0,    0, 0, 0,     0,   0, 0],
            [0, 0, 0,    0, 0, 0,     0,   0, 0],
            [0, 0, 0,    0, 0, 0,     0,   0, 0],

        ]
        )
    
    eAT = smp.exp(A*T_LOOKAHEAD)

    B = smp.Matrix(
        [
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],

            [1/MASS, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
        ]
        )

    A = np.array(A).astype(np.float64)
    int_eAT = np.zeros_like(A)
    rowcol = int_eAT.shape[0]
    for row in range(rowcol):
        for column in range(rowcol):
            f = lambda x: sp_linalg.expm(A*(T_LOOKAHEAD-x))[row,column]
            int_eAT[row,column] = sp_int.quad(f, 0, T_LOOKAHEAD)[0]


    int_eATB = int_eAT @ B
    eAT = np.array(eAT).astype(np.float64)
    int_eATB = np.array(int_eATB).astype(np.float64)

    # print(f"A: \n {A}")
    # print(f"eAT: \n {eAT}")
    # print(f"int_eAT: \n {int_eAT}")

    # print(f"B: \n {B}")
    # print(f"int_eATB: \n {int_eATB}")
    # print(f"C: \n {C}")

    # jac_inv = np.linalg.inv(getyorai_gJac_linear_predict()) #Calculate Inverse Jacobian of Linearized Model Matrices
    jac_inv = C @ int_eATB
    return eAT, int_eATB, int_eAT, jac_inv

In [80]:
C = observer_matrix()
eAT, int_eATB, int_eAT, jac_inv = linearized_model()

In [81]:
gravity = np.array([[MASS * GRAVITY, 0, 0, 0]]).T #gravity vector that counteracts input vector: [-mg, 0, 0, 0]
print(f"{gravity = }")
input = np.array([[-16., 0., 0., 0.]]).T #input vector: [F, Mx, My, Mz]
print(f"{input = }")
print(f"{x = }")
# unforced = self.eAT @ self.state_vector
# print(f"{unforced = }")
# forced = self.int_eATB @ (curr_input + gravity)
# print(f"{forced = }")
lin_pred = eAT@x + int_eATB @ (input + gravity) # xdot = eAT*x(t) + int_eATB*(u-gravity)
yorai_g = C @ lin_pred # y(t) = C*x(t) = [x,y,z,yaw]
yorai_g


gravity = array([[14.709],
       [ 0.   ],
       [ 0.   ],
       [ 0.   ]])
input = array([[-16.],
       [  0.],
       [  0.],
       [  0.]])
x = array([[ 0. ],
       [ 0. ],
       [-0.5],
       [ 0. ],
       [ 0. ],
       [ 0. ],
       [ 0. ],
       [ 0. ],
       [ 0. ]])


array([[ 0.        ],
       [ 0.        ],
       [-0.77541333],
       [ 0.        ]])

In [82]:
jac_inv

array([[ 0.        ,  0.        , -0.83677867,  0.        ],
       [ 0.        ,  0.83677867,  0.        ,  0.        ],
       [ 0.21333333,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.8       ]])